# Example how to query multiple items using paging

In [1]:
from sgqlc.operation import Operation
from pdp_schema import pdp_schema as schema
from pdp_graphql_client_python import client

In [15]:
! echo "${PDP_API}"

https://graphql-api.pdp.int.srgssr.ch/graphql


In [16]:
# show available queries
# *Page queries allow for paging (https://graphql.org/learn/pagination/)
# first=[number of queries to retrieve, e.g. *first* 10 queries]
# after=[id or cursor position from which to start]
schema.Query

type Query {
  faroProgramPage(first: Int!, after: String): FaroProgramPage!
  faroPrograms(ids: [String!]!): [FaroProgram]!
  randomFaroPrograms: [FaroProgram!]!
  faroItemPage(first: Int!, after: String): FaroItemPage!
  faroItems(ids: [String!]!): [FaroItem]!
  faroItemsByPlayUrn(urns: [String!]!): [FaroItem]!
  faroItemsByPlayUrnGraphDb(urns: [String!]!): [FaroItem]!
  randomFaroItems: [FaroItem!]!
}

In [46]:
# retrieve query operations
op = Operation(schema.Query)

In [33]:
type(op)

sgqlc.operation.Operation

In [19]:
# return current query (empty)
op

query {
}

In [47]:
# select query type and configure with argument
faro = op.faro_program_page(first=10)

In [48]:
# return current query (empty)
op

query {
  faroProgramPage(first: 10) {
    edges {
      id
      bu
      mediaType
      episodeIds
      sTit
      sDatStart
      sGef
      pTit
      department
      workgroup
      seriesNr
    }
    cursor
  }
}

In [54]:
# select fields to be retured
faro.edges.media_type()
faro.edges.s_dat_start()
faro.edges.s_gef()
faro.edges.s_tit()

sTit

In [55]:
# return current query
op

query {
  faroProgramPage(first: 10) {
    edges {
      mediaType
      sDatStart
      sGef
      sTit
    }
  }
}

In [56]:
# run query and retrieve data
data = client.run_query(op)

In [57]:
# display raw data (json)
data

{'data': {'faroProgramPage': {'edges': [{'mediaType': 'audio',
     'sDatStart': '1994-06-26',
     'sGef': 'Doppelpunkt',
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2012-11-26',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2006-11-21',
     'sGef': 'Oz',
     'sTit': 'Contribuziun'},
    {'mediaType': 'audio',
     'sDatStart': '2012-09-28',
     'sGef': None,
     'sTit': None},
    {'mediaType': None, 'sDatStart': None, 'sGef': None, 'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2018-04-21',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '1979-03-20',
     'sGef': 'Wir lesen vor',
     'sTit': 'Fortsetzung folgt: Eine Krähe war mit mir\n'},
    {'mediaType': 'audio',
     'sDatStart': '2009-06-18',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2005-04-20',
     'sGef': 'Kulturtipp',
     'sTit': None},
    {'mediaType': 'audio',
  

In [58]:
# convert again to object oriented results
results = (op + data).faro_program_page

In [59]:
# display object transformed data
results

FaroProgramPage(edges=[FaroProgram(media_type='audio', s_dat_start='1994-06-26', s_gef='Doppelpunkt', s_tit=None), FaroProgram(media_type='audio', s_dat_start='2012-11-26', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2006-11-21', s_gef='Oz', s_tit='Contribuziun'), FaroProgram(media_type='audio', s_dat_start='2012-09-28', s_gef=None, s_tit=None), FaroProgram(media_type=None, s_dat_start=None, s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2018-04-21', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='1979-03-20', s_gef='Wir lesen vor', s_tit='Fortsetzung folgt: Eine Krähe war mit mir\n'), FaroProgram(media_type='audio', s_dat_start='2009-06-18', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2005-04-20', s_gef='Kulturtipp', s_tit=None), FaroProgram(media_type='audio', s_dat_start='2002-01-28', s_gef='Regionaljournal Zürich Schaffhausen', s_tit=None)])

In [63]:
# discover data types
type(results.edges[0])

type FaroProgram {
  id: String!
  bu: String
  mediaType: String
  episodeIds: [String!]!
  sTit: String
  sDatStart: String
  sGef: String
  pTit: String
  department: String
  workgroup: String
  seriesNr: Int
  moderators: [FaroModerator!]!
  itemPage(first: Int!, after: String): FaroItemPage!
  playEpisodes: [PlayEpisode!]!
}

In [69]:
from collections import Counter
from datetime import date

# count year occurrences
Counter(date.fromisoformat(x.s_dat_start).year for x in results.edges if x.s_dat_start)

Counter({1994: 1,
         2012: 2,
         2006: 1,
         2018: 1,
         1979: 1,
         2009: 1,
         2005: 1,
         2002: 1})